In [ ]:
from google.colab import drive 
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import json 
import matplotlib.pyplot as plt 


In [ ]:
path = '/content/drive/MyDrive/Robosoft-ML/LSTM/new_datav4.json'



with open(path) as file : 
  data = json.load(file)


training_sentences = [] 
training_labels = [] 
labels = [] 
responses = [] 

for intent in data['intents'] : 
  for pattern in intent['patterns']:
    training_sentences.append(pattern) 
    training_labels.append(intent['tag'])
  responses.append(intent['responses']) 

  labels.append(intent['tag'])

num_classes = len(labels)


In [ ]:
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
label_encoder = LabelEncoder()
integer_encoded = label_encoder.fit_transform(training_labels)
onehot_encoder = OneHotEncoder(sparse=False)
integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)
onehot_encoded = onehot_encoder.fit_transform(integer_encoded)
print(onehot_encoded)

[[0. 1. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 1. 0. 0.]
 [0. 0. 1. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0.

In [ ]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

In [ ]:
vocab_size = 1000 
embedding_dim = 8 
max_len = 10 
oov_token = "<OOV>"

tokenizer = Tokenizer(num_words = vocab_size , oov_token = oov_token , lower = True , filters='!"#$%&()*+,-./:;<=>@[\\]^_`{|}~\t\n' , split = ' ' )

In [ ]:
tokenizer.fit_on_texts(list(training_sentences)) 

In [ ]:
list_tokenized_train = tokenizer.texts_to_sequences(training_sentences)

In [ ]:
max_len = 10 
sequences = tokenizer.texts_to_sequences(training_sentences)
padded_sequences = pad_sequences(sequences, truncating='post', padding = 'post', maxlen=max_len)


In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(padded_sequences, onehot_encoded, test_size = 0.3, shuffle = True, stratify=onehot_encoded)

In [ ]:
print("Train Dimensions : " ,X_train.shape)
print("Test Dimensions : " ,X_test.shape)


Train Dimensions :  (54, 10)
Test Dimensions :  (24, 10)


In [ ]:
from keras.layers import Dense, Input, LSTM, Embedding, Dropout, Activation
from keras.models import Model
from keras.layers import Bidirectional, GlobalMaxPool1D




In [ ]:
#input layer 
inp = Input(shape = (max_len , ))
max_features = vocab_size
#embedding layer
embed = embedding_dim
x = Embedding(max_features , embed , mask_zero = True)(inp)
#lstm layer 
x = LSTM(16, return_sequences=True,name='lstm_layer')(x)
#max pooling for reducing dimensions 
x = GlobalMaxPool1D()(x)
#dropout layer
x = Dropout(0.25)(x)
#dense layer
x = Dense(16 , activation = "relu")(x)
x = Dense(8 , activation = "tanh")(x)
x = Dense(6 , activation = "softmax")(x)



















In [ ]:
model = Model(inputs=inp, outputs=x)
model.compile(loss='categorical_crossentropy',
                  optimizer='adam',
                  metrics=['accuracy'])

In [ ]:
model.summary()

Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_4 (InputLayer)        [(None, 10)]              0         
                                                                 
 embedding_3 (Embedding)     (None, 10, 8)             8000      
                                                                 
 lstm_layer (LSTM)           (None, 10, 16)            1600      
                                                                 
 global_max_pooling1d_3 (Glo  (None, 16)               0         
 balMaxPooling1D)                                                
                                                                 
 dropout_6 (Dropout)         (None, 16)                0         
                                                                 
 dense_9 (Dense)             (None, 16)                272       
                                                           

In [ ]:
from keras.callbacks import EarlyStopping
from keras.callbacks import ModelCheckpoint

In [ ]:
callback = EarlyStopping(monitor = 'val_loss' , patience = 10 , mode = 'min' )

In [ ]:
filepath = '/content/drive/MyDrive/Robosoft-ML/LSTM/bestv4.h5'
checkpoint = ModelCheckpoint(filepath, monitor='val_accuracy', verbose=1, save_best_only=True, mode = 'max')

callback_list = [checkpoint ]

In [ ]:
epochs = 500
history=model.fit(X_train, y_train, batch_size=8, epochs=epochs,validation_data=(X_test,y_test) , callbacks = callback_list)

In [ ]:
import pickle

# to save the fitted tokenizer
with open('/content/drive/MyDrive/Robosoft-ML/LSTM/tokenizer.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)
    
# to save the fitted label encoder
with open('/content/drive/MyDrive/Robosoft-ML/LSTM/label_encoder.pickle', 'wb') as ecn_file:
    pickle.dump(onehot_encoded, ecn_file, protocol=pickle.HIGHEST_PROTOCOL)

with open('/content/drive/MyDrive/Robosoft-ML/LSTM/onehot_encoder.pickle', 'wb') as ecn_file:
    pickle.dump(integer_encoded, ecn_file, protocol=pickle.HIGHEST_PROTOCOL)

with open('/content/drive/MyDrive/Robosoft-ML/LSTM/lbl_encoder.pickle', 'wb') as ecn_file:
    pickle.dump(label_encoder, ecn_file, protocol=pickle.HIGHEST_PROTOCOL)


hello
ChatBot: Hello
i want to be a part of trf
ChatBot: 
Every year TRF conducts its recruitment process around the end of academic year. This process is for Mechanical , Electrical , Programming as well as the Admin team. The details about the entire process is informed well before the registration and ample preparation time is provided to the students,. 


KeyboardInterrupt: ignored

0

[0.018011774867773056,
 0.011098857037723064,
 0.018163669854402542,
 0.9429423213005066,
 0.0003300470416434109,
 0.009453355334699154]